# Segundo Punto

Se usara el algoritmo de clustering jerárquico con los diferentes criterios de enlazamiento (linkage) para agrupar los datos. Finalmente,usando el criterio de agrupamiento J4, analizará los diferentes puntos de corte para el agrupamiento. ¿Qué puede concluir?